In [ ]:
!pip install datasets torch transformers
!wget http://www.cs.cornell.edu/~cristian/data/cornell_movie_dialogs_corpus.zip
!unzip -qq cornell_movie_dialogs_corpus.zip
!rm cornell_movie_dialogs_corpus.zip
!mkdir datasets
!mv cornell\ movie-dialogs\ corpus/movie_conversations.txt ./datasets
!mv cornell\ movie-dialogs\ corpus/movie_lines.txt ./datasets

In [ ]:
from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.utils.data
import math
import torch.nn.functional as F

# 1) Data Processing

In [ ]:
# data processing
max_len = 25

def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines()
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

# extract text
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

# generate question answer pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i == len(ids) - 1:
            break
        
        first = remove_punc(lines_dic[ids[i]].strip())      
        second = remove_punc(lines_dic[ids[i+1]].strip())
        qa_pairs.append(first.split()[:max_len])
        qa_pairs.append(second.split()[:max_len])
        pairs.append(qa_pairs)

# sample
print(pairs[20])

[['i', 'really', 'really', 'really', 'wanna', 'go', 'but', 'i', 'cant', 'not', 'unless', 'my', 'sister', 'goes'], ['im', 'workin', 'on', 'it', 'but', 'she', 'doesnt', 'seem', 'to', 'be', 'goin', 'for', 'him']]


In [ ]:
# word map
min_word_freq = 5

word_freq = Counter()
for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])

words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

print("Total words are: {}".format(len(word_map)))

# encode sentences based on word map
def encode_question(words, word_map):
    enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

def encode_reply(words, word_map):
    enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + \
    [word_map['<end>']] + [word_map['<pad>']] * (max_len - len(words))
    return enc_c

pairs_encoded = []
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append([qus, ans])

Total words are: 18243


In [ ]:
# dataset and dataloader
class Dataset(Dataset):

    def __init__(self, pairs):

        self.pairs = pairs
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
        return question, reply

    def __len__(self):
        return self.dataset_size

train_loader = DataLoader(Dataset(pairs_encoded), batch_size=32, shuffle=True, pin_memory=True)
question, reply = next(iter(train_loader))
print("Question: ", question.size())
print("Answer: ", reply.size())

Question:  torch.Size([32, 25])
Answer:  torch.Size([32, 27])


# 2) Masking

In [ ]:
# create mask
def create_masks(question, reply_input, reply_target, device='cpu'):
    
    def subsequent_mask(size):
        # (max_words, max_words)
        # binary triangle
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        # (1. max_words, max_words)
        return mask.unsqueeze(0)
    
    # boolean(m, max_words)
    question_mask = question != 0

    # (m, 1, 1, max_words)
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)         
    
    # boolean(m, max_words)
    reply_input_mask = reply_input != 0
    # (m, 1, max_words)
    reply_input_mask = reply_input_mask.unsqueeze(1)  

    # only include triangle and non-pad token
    # (m, max_words, max_words)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data) 

    # (batch_size, 1, max_words, max_words)
    reply_input_mask = reply_input_mask.unsqueeze(1) 

    # (batch_size, max_words)
    reply_target_mask = reply_target != 0              
    
    return question_mask, reply_input_mask, reply_target_mask

reply_input = reply[:, :-1]
reply_target = reply[:, 1:]
print('Reply Target Size: ', reply_target.size())

# Create mask and add dimensions
question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)
print('question_mask Size: ', question_mask.size())
print('reply_input_mask Size: ', reply_input_mask.size())
print('reply_target_mask Size: ', reply_target_mask.size())

Reply Target Size:  torch.Size([32, 26])
question_mask Size:  torch.Size([32, 1, 1, 25])
reply_input_mask Size:  torch.Size([32, 1, 26, 26])
reply_target_mask Size:  torch.Size([32, 26])


# 3) Embedding

In [ ]:
# Positional Embedding
class Embeddings(nn.Module):
    """
    Implements embeddings of the words and adds their positional encodings. 
    """
    def __init__(self, vocab_size, d_model, max_len=50, num_layers=6):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size, d_model)
        # (1, max_len, d_model)
        self.pe = self.create_positional_encoding(max_len, self.d_model)     
        # (1, num_layers, d_model)
        self.te = self.create_positional_encoding(num_layers, self.d_model)  
        self.dropout = nn.Dropout(0.1)
        
    def create_positional_encoding(self, max_len, d_model, device='cpu'):
        pe = torch.zeros(max_len, d_model).to(device)
        # for each position of the word
        for pos in range(max_len):   
            # for each dimension of the each position
            for i in range(0, d_model, 2):   
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        # include the batch size
        pe = pe.unsqueeze(0)   
        return pe
        
    def forward(self, embedding, layer_idx):
        # create embed weight during first layer
        # (m, max_len) --> (m, max_len, d_model)
        if layer_idx == 0:
            embedding = self.embed(embedding) * math.sqrt(self.d_model)

        ### Positional Embedding

        # pe will automatically be expanded with the same batch size as encoded_words
        # (m, max_len, d_model)
        embedding += self.pe[:, :embedding.size(1)]

        # te: (1, num_layers, d_model) --> (1, 1, d_model) --> (1, max_len, d_model)
        # (m, max_len, d_model)
        embedding += self.te[:, layer_idx, :].unsqueeze(1).repeat(1, embedding.size(1), 1)
        embedding = self.dropout(embedding)
        return embedding

embed_model = Embeddings(len(word_map), 512)
result = embed_model.forward(question, 0)
print(result.size())

torch.Size([32, 25, 512])


<img src='https://lilianweng.github.io/posts/2020-04-07-the-transformer-family/transformer.png'>

<img src='https://1.bp.blogspot.com/-AVGK0ApREtk/WaiAuzddKVI/AAAAAAAAB_A/WPV5ropBU-cxrcMpqJBFHg73K9NX4vywwCLcBGAs/s1600/image2.png'>

# 4) Transformers

In [ ]:
### transformers
class MultiHeadAttention(nn.Module):
    
    def __init__(self, heads, d_model, dropout=0.1):
        
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(dropout)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.output_linear = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask):
        """
        query, key, value of shape: (batch_size, max_len, d_model)
        mask of shape: (batch_size, 1, 1, max_words)
        """
        # (batch_size, max_len, d_model)
        query = self.query(query)
        key = self.key(key)        
        value = self.value(value)   
        
        # (batch_size, max_len, d_model) --> (batch_size, max_len, h, d_k) --> (batch_size, h, max_len, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)   
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)  
        
        # (batch_size, h, max_len, d_k) matmul (batch_size, h, d_k, max_len) --> (batch_size, h, max_len, max_len)
        scores = torch.matmul(query, key.permute(0, 1, 3, 2)) / math.sqrt(query.size(-1))

        # fill 0 mask with super small number so it wont affect the softmax weight
        # (batch_size, h, max_len, max_len)
        scores = scores.masked_fill(mask == 0, -1e9)    

        # (batch_size, h, max_len, max_len)
        # softmax to put attention weight for all non-pad tokens
        # max_len X max_len matrix of attention
        weights = F.softmax(scores, dim=-1)           
        weights = self.dropout(weights)

        # (batch_size, h, max_len, max_len) matmul (batch_size, h, max_len, d_k) --> (batch_size, h, max_len, d_k)
        context = torch.matmul(weights, value)

        # (batch_size, h, max_len, d_k) --> (batch_size, max_len, h, d_k) --> (batch_size, max_len, d_model)
        context = context.permute(0, 2, 1, 3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)

        # (batch_size, max_len, d_model)
        return self.output_linear(context)

class FeedForward(nn.Module):

    def __init__(self, d_model, middle_dim=2048):
        super(FeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)
        self.activation = torch.nn.GELU()

    def forward(self, x):
        out = self.activation(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

class EncoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, mask):
        # embeddings: (batch_size, max_len, d_model)
        # encoder mask: (batch_size, 1, 1, max_len)
        # result: (batch_size, max_len, d_model)
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        # residual layer
        interacted = self.layernorm(interacted + embeddings)
        # bottleneck
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

class DecoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, encoded, src_mask, target_mask):
        
        # embeddings: (batch_size, max_len, d_model)
        # decoder mask: (m, 1, max_len, max_len)
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)

        # (batch_size, max_len, d_model)
        # encoder-decoder mask: (m, 1, 1, max_len)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded

class Transformer(nn.Module):
    
    def __init__(self, d_model, heads, num_layers, word_map):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.num_layers = num_layers
        self.vocab_size = len(word_map)
        self.embed = Embeddings(self.vocab_size, d_model, num_layers=num_layers)
        self.encoder = EncoderLayer(d_model, heads) 
        self.decoder = DecoderLayer(d_model, heads)
        self.logit = nn.Linear(d_model, self.vocab_size)
        
    def encode(self, src_embeddings, src_mask):
        for i in range(self.num_layers):
            src_embeddings = self.embed(src_embeddings, i)
            src_embeddings = self.encoder(src_embeddings, src_mask)
        return src_embeddings
    
    def decode(self, tgt_embeddings, target_mask, src_embeddings, src_mask):
        for i in range(self.num_layers):
            tgt_embeddings = self.embed(tgt_embeddings, i)
            tgt_embeddings = self.decoder(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
        
    def forward(self, src_words, src_mask, target_words, target_mask):
        # (m, max_len) --> (m, max_len, d_model)
        encoded = self.encode(src_words, src_mask)
        # (m, max_len, d_model) --> (m, max_len, d_model)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        # (m, max_len, vocab_size)
        out = F.log_softmax(self.logit(decoded), dim=2)
        return out

# 5) Optimizer

<img src='https://miro.medium.com/max/886/1*ZhGLUwaaqlJ9C0WK0nbAEA.png'>



## Relationship between Entropy, Cross-Entropy, and KL-Divergence

https://towardsdatascience.com/entropy-cross-entropy-and-kl-divergence-explained-b09cdae917a

In [ ]:
# optimizer & loss
class AdamWarmup:
    
    def __init__(self, model_size, warmup_steps, optimizer):
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
        
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
        
    def step(self):
        # Increment the number of steps each time we call the step function
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        # update the learning rate
        self.lr = lr
        self.optimizer.step()

class LossWithLS(nn.Module):

    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size
        
    def forward(self, prediction, target, mask):
        """
        prediction of shape: (batch_size, max_words, vocab_size)
        target and mask of shape: (batch_size, max_words)
        """

        # (batch_size * max_words, vocab_size)
        prediction = prediction.view(-1, prediction.size(-1))   

        # (batch_size * max_words)
        target = target.contiguous().view(-1)   
        mask = mask.float()
        mask = mask.view(-1)       

        # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)

        # (batch_size * max_words, vocab_size)
        loss = self.criterion(prediction, labels)    
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss

### testing
transformer = Transformer(d_model=512, heads=8, num_layers=2, word_map=word_map)
transformer = transformer.to('cpu')
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size=512, warmup_steps=4000, optimizer=adam_optimizer)
criterion = LossWithLS(len(word_map), 0.2)
out = transformer(question, question_mask, reply_input, reply_input_mask)
print(out.size())

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


torch.Size([32, 26, 18243])


In [ ]:
### training step
def train(train_loader, transformer, criterion, epoch, device='cpu'):
    
    transformer.train()
    sum_loss = 0
    count = 0

    for i, (question, reply) in enumerate(train_loader):
        
        samples = question.shape[0]

        # Move to device
        question = question.to(device)
        reply = reply.to(device)

        # Prepare Target Data
        reply_input = reply[:, :-1]
        reply_target = reply[:, 1:]

        # Create mask and add dimensions
        question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)

        # Get the transformer outputs
        out = transformer(question, question_mask, reply_input, reply_input_mask)

        # Compute the loss
        loss = criterion(out, reply_target, reply_target_mask)
        
        # Backprop
        transformer_optimizer.optimizer.zero_grad()
        loss.backward()
        transformer_optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        if i % 100 == 0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

In [ ]:
# evaluation
def evaluate(transformer, question, question_mask, max_len, word_map, device="cpu"):
    """
    Performs Greedy Decoding with a batch size of 1
    """
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['<start>']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim=1)
        next_word = next_word.item()
        if next_word == word_map['<end>']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim=1)
        
    # Construct Sentence
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words if w not in {word_map['<start>']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

device = 'cpu'
while(1):
    question = input("Question: ") 
    if question == 'quit':
        break
    max_len = input("Maximum Reply Length: ")
    enc_qus = [word_map.get(word, word_map['<unk>']) for word in question.split()]
    question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
    question_mask = (question!=0).to(device).unsqueeze(1).unsqueeze(1)  
    sentence = evaluate(transformer, question, question_mask, int(max_len), word_map)
    print(sentence)

Question: what is your name
Maximum Reply Length: 25
begged funky begged funky begged funky begged funky begged funky begged funky begged funky begged funky begged funky begged funky begged funky porters fiction
Question: quit


# 6) Pretrained Transformer

In [ ]:
# dataset and dataloader
from transformers import BartTokenizer, BartModel
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
max_len = 25
class Dataset(Dataset):

    def __init__(self, pairs):

        self.pairs = pairs
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):

        question = tokenizer(" ".join(self.pairs[i][0]), max_length=max_len, truncation=True, padding="max_length", return_tensors="pt")
        reply = tokenizer(" ".join(self.pairs[i][1]), max_length=max_len, truncation=True, padding="max_length", return_tensors="pt")
        
        return question['input_ids'][0], question['attention_mask'][0], reply['input_ids'][0], reply['attention_mask'][0]

    def __len__(self):
        return self.dataset_size

train_loader = DataLoader(Dataset(pairs), batch_size=32, shuffle=True, pin_memory=True)
question, q_mask, reply, t_mask = next(iter(train_loader))
print("Question: ", question.size())
print("Answer: ", reply.size())

Question:  torch.Size([32, 25])
Answer:  torch.Size([32, 25])


In [ ]:
class Transformers(nn.Module):

    def __init__(self, num_classes, hidden_dim=512, nheads=8,
                 num_encoder_layers=6, num_decoder_layers=6):
        super().__init__()

        # create a default PyTorch transformer
        self.backbone = BartModel.from_pretrained("facebook/bart-base")
        self.transformer = nn.Transformer(
            hidden_dim, nheads, num_encoder_layers, num_decoder_layers)
        self.linear_class = nn.Linear(hidden_dim, num_classes)

    def forward(self, src, tgt, src_mask, tgt_mask):

        src_embed = self.backbone(src, src_mask)['encoder_last_hidden_state'].permute(1, 0, 2)
        tgt_embed = self.backbone(tgt, tgt_mask)['encoder_last_hidden_state'].permute(1, 0, 2)

        out = self.transformer(
            src=src_embed, 
            tgt=tgt_embed, 
            src_key_padding_mask=src_mask, 
            tgt_key_padding_mask=tgt_mask)
        
        result = self.linear_class(out)
        result = F.log_softmax(result, dim=2)
        return result

model = Transformers(len(word_map), hidden_dim=768)
result = model(question, q_mask, reply, t_mask)
print(result.size())